# LOFAR conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.35 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="small_lofar.ms")

[2024-08-26 13:46:15,164]  WARNING  graphviper:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-26 13:46:15,165]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List   
 ──────────────── 
  small_lofar.ms

[2024-08-26 13:46:16,401]     INFO  graphviper:  File exists: small_lofar.ms 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "small_lofar.ms"
main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "small_lofar.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-08-26 13:46:16,601]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-26 13:46:16,604]     INFO  graphviper:  Number of partitions: 1 
[2024-08-26 13:46:16,604]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [0] 
[2024-08-26 13:46:16,705]  WARNING  graphviper:  Source_id is -1. No source information will be included in the field_and_source_xds. 


## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,small_lofar_0,[],"(5, 2775, 15, 4)","[XX, XY, YX, YY]",[0],SB-349_0,[BEAM_4_0],[Unknown],[],"[fk5, 14h00m00.00s, 86d00m00.00s]",1.270615e+08,1.272324e+08


In [5]:
ps.get(0)

<xarray.Dataset> Size: 12MB
Dimensions:                     (time: 5, baseline_id: 2775, frequency: 15,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    antenna_id                  (baseline_id) int32 11kB dask.array<chunksize=(2775,), meta=np.ndarray>
    antenna_name                (baseline_id) <U9 100kB dask.array<chunksize=(2775,), meta=np.ndarray>
    baseline_antenna1_name      (baseline_id) <U9 100kB dask.array<chunksize=(2775,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U9 100kB dask.array<chunksize=(2775,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 22kB 0 1 2 ... 2773 2774
  * frequency                   (frequency) float64 120B 1.271e+08 ... 1.272e+08
    mount                       (baseline_id) <U6 67kB dask.array<chunksize=(2775,), meta=np.ndarray>
  * polarization                (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
    scan_number                 (time) int64 40B dask.array<chunksize=(5,), meta=np.ndarray>
    station                     (baseline_id) <U5 56kB dask.array<chunksize=(2775,), meta=np.ndarray>
  * time                        (time) float64 40B 1.52e+09 ... 1.52e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 111kB dask.array<chunksize=(5, 2775), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 832kB dask.array<chunksize=(5, 2775, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 111kB dask.array<chunksize=(5, 2775), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 333kB dask.array<chunksize=(5, 2775, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 7MB dask.array<chunksize=(5, 2775, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 3MB dask.array<chunksize=(5, 2775, 1, 4), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_name': ['BEAM_4_0'], 'line_name': [], 'num_lines...
    type:            visibility
    pointing_xds:    <xarray.Dataset> Size: 3kB\nDimensions:        (time: 1,...
    antenna_xds:     <xarray.Dataset> Size: 14kB\nDimensions:                ...

In [6]:
ps.get(0).attrs["antenna_xds"]

<xarray.Dataset> Size: 14kB
Dimensions:                (antenna_name: 74, cartesian_pos_label: 3,
                            receptor_label: 2, sky_dir_label: 2)
Coordinates:
  * antenna_name           (antenna_name) <U9 3kB 'CS001HBA0' ... 'IE613HBA'
  * cartesian_pos_label    (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                  (antenna_name) <U6 2kB dask.array<chunksize=(74,), meta=np.ndarray>
    polarization_type      (antenna_name, receptor_label) <U1 592B dask.array<chunksize=(74, 2), meta=np.ndarray>
  * receptor_label         (receptor_label) <U5 40B 'pol_0' 'pol_1'
  * sky_dir_label          (sky_dir_label) <U3 24B 'ra' 'dec'
    station                (antenna_name) <U5 1kB dask.array<chunksize=(74,), meta=np.ndarray>
Data variables:
    ANTENNA_DISH_DIAMETER  (antenna_name) float64 592B dask.array<chunksize=(74,), meta=np.ndarray>
    ANTENNA_FEED_OFFSET    (antenna_name, cartesian_pos_label) float64 2kB dask.array<chunksize=(74, 3), meta=np.ndarray>
    ANTENNA_POSITION       (antenna_name, cartesian_pos_label) float64 2kB dask.array<chunksize=(74, 3), meta=np.ndarray>
    BEAM_OFFSET            (antenna_name, receptor_label, sky_dir_label) float64 2kB dask.array<chunksize=(74, 2, 2), meta=np.ndarray>
    RECEPTOR_ANGLE         (antenna_name, receptor_label) float64 1kB dask.array<chunksize=(74, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  LOFAR
    relocatable_antennas:    False